In [3]:
!pip install langchain PyPDF2 sentence-transformers faiss-cpu gpt4all langchain_community torch

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 232.6/232.6 kB 7.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 30.7/30.7 MB 53.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 121.6/121.6 MB 7.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.5/2.5 MB 68.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 3.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 49.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 14.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 34.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 2.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 4.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 9.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 8.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207

In [1]:
from google.colab import drive
drive.mount('/content/drive/')

Mounted at /content/drive/


In [2]:
WORKING_PATH = '/content/drive/MyDrive/Colab Notebooks/AgentesIA'
%cd {WORKING_PATH}

import sys
sys.path.append(WORKING_PATH)

/content/drive/MyDrive/Colab Notebooks/AgentesIA


In [4]:
import os
from PyPDF2 import PdfReader
from tqdm import tqdm
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain_community.embeddings import HuggingFaceBgeEmbeddings
from langchain_community.vectorstores import FAISS
from gpt4all import GPT4All
import torch
from huggingface_hub import hf_hub_download

In [5]:
if torch.cuda.is_available():
    print("¡GPU detectada!:", torch.cuda.get_device_name(0))
else:
    print("No se encontró GPU. Se usará CPU.")

No se encontró GPU. Se usará CPU.


In [6]:
data_directory = "/content/drive/MyDrive/Colab Notebooks/AgentesIA/data"
model_directory = "/content/drive/MyDrive/Colab Notebooks/AgentesIA/gpt4all_models"
model_name = "mistral-7b-instruct-v0.1.Q4_K_M.gguf"
repo_id = "TheBloke/Mistral-7B-Instruct-v0.1-GGUF"
model_path = os.path.join(model_directory, model_name)

In [ ]:
if not os.path.exists(model_path):
    os.makedirs(model_directory, exist_ok=True)
    model_path = hf_hub_download(
        repo_id=repo_id,
        filename=model_name,
        local_dir=model_directory
    )

llm = GPT4All(model_path)

In [ ]:
embedder = HuggingFaceBgeEmbeddings(model_name="sentence-transformers/paraphrase-multilingual-MiniLM-L12-v2")

In [8]:
def read_pdf(file_path):
    pdf_reader = PdfReader(file_path)
    text = "\n".join([page.extract_text() for page in pdf_reader.pages if page.extract_text()])
    return text

pdf_files = [f for f in os.listdir(data_directory) if f.endswith(".pdf")]

documents = []
for pdf_file in tqdm(pdf_files, desc="Procesando PDFs"):
    pdf_text = read_pdf(os.path.join(data_directory, pdf_file))
    text_splitter = RecursiveCharacterTextSplitter(chunk_size=500, chunk_overlap=50)
    chunks = text_splitter.split_text(pdf_text)
    documents.extend(chunks)

if documents:
    vectorstore = FAISS.from_texts(documents, embedder)
else:
    raise ValueError("No se encontraron documentos para indexar.")

def find_most_relevant_document(question):
    docs = vectorstore.similarity_search(question, k=3)
    return "\n".join([doc.page_content for doc in docs])

<ipython-input-8-0ab26f1393c4>:1: LangChainDeprecationWarning: The class `HuggingFaceBgeEmbeddings` was deprecated in LangChain 0.2.2 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-huggingface package and should be used instead. To use it run `pip install -U :class:`~langchain-huggingface` and import as `from :class:`~langchain_huggingface import HuggingFaceEmbeddings``.
  embedder = HuggingFaceBgeEmbeddings(model_name="sentence-transformers/paraphrase-multilingual-MiniLM-L12-v2")
/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to a

modules.json:   0%|          | 0.00/229 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/122 [00:00<?, ?B/s]

README.md:   0%|          | 0.00/4.12k [00:00<?, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/53.0 [00:00<?, ?B/s]

...

tokenizer_config.json:   0%|          | 0.00/480 [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/9.08M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/239 [00:00<?, ?B/s]

1_Pooling%2Fconfig.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

Procesando PDFs: 100%|██████████| 3/3 [00:02<00:00,  1.07it/s]


In [9]:
while True:
    pregunta = input("Introduce tu pregunta sobre los documentos PDF (o escribe 'salir' para terminar): ")
    if pregunta.lower() in ['salir', 'exit', 'quit']:
        break

    contexto = find_most_relevant_document(pregunta)

    prompt = f"Contexto: {contexto}\n\nPregunta: {pregunta}\n\nRespuesta:"
    response = llm.generate(prompt, max_tokens=1000)

    print(response)

Introduce tu pregunta sobre los documentos PDF (o escribe 'salir' para terminar): servicios training2works

La
compañía
training2works
ofrece
servicios
de
formación
digital
orientados
a
la
formación
profesional.
Los
servicios
se
basan
en
prácticas
realizadas
por
los
estudiantes
donde
aprenden
conocimientos
relacionados
con
el
mundo
digital.
Después
de
la
formación,
los
estudiantes
se
buscan
un
trabajo
en
una
empr esa
cliente
ﬁnal.
La
compañía
ofrece
servicios
personalizados
basados
en
el
perfil
del
estudiante,
la
experiencia
de
los
profesionales
y
los
resultados
anteriormente
obtenidos.
La
compañía
ofrece
servicios
remotos
o
en
el
menor
tiempo
posible,
lo
que
hace
más
fácil
para
los
estudiantes
realizar
la
formación.
Introduce tu pregunta sobre los documentos PDF (o escribe 'salir' para terminar): cuales son los pasos del flujo de trabajo
 1, 2, 3, 4, 5, 6, 7, 8
Introduce tu pregunta sobre los documentos PDF (o escribe 'salir' para terminar): cual es el plan de accion para la clinica b